<a href="https://colab.research.google.com/github/endophenotype/Spark/blob/main/Spark_%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Параллельная коллекция, содержащая числа от 1
до 5:

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = spark.sparkContext
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(distData)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


После создания распределенный набор данных ( distData) может работать
параллельно. Например, мы можем вызвать distData.reduce(lambda a, b: a + b),
чтобы сложить элементы списка.

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[1]'
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
summ = distData.reduce(lambda a, b: a + b)
print(summ)

15


Текстовые файлы RDD могут быть созданы с использованием
SparkContextметода textFile. Этот метод принимает URI для файла (либо
локальный путь на машине, либо URI hdfs://, s3a://, и т. д.) и считывает его как
набор строк. Вот пример вызова:

In [ ]:
distFile = sc.textFile("/content/G.txt")
print(distFile)

/content/G.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0


Как и текстовые файлы, SequenceFiles можно сохранять и загружать, указав
путь.

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x))
sorted(sc.sequenceFile("/content/A/part-00000").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

По умолчанию каждый преобразованный RDD может пересчитываться
каждый раз, когда над ним запускается действие. Однако можно
сохранить RDD в памяти с помощью метода persist(или cache), и в этом случае
Spark сохранит элементы в кластере для более быстрого доступа при
следующем запросе. Также поддерживается сохранение RDD на диске или их
репликация на нескольких узлах.


In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
lines = sc.textFile("/content/G.txt")
lineLengths = lines.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
print(totalLength)

1109


Лямбда-выражения для простых функций, которые можно записать в виде
выражения. (Лямбды не поддерживают функции с несколькими операторами
или операторы, которые не возвращают значение.)

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
words = ["hello this is line one", "hello this is line two"]
words_rdd = sc.parallelize(words)
print (words_rdd.collect())
words_rdd = words_rdd.flatMap(lambda line: line.split(" "))
print (words_rdd.collect())
pairs = words_rdd.map(lambda s: (s, 1))
print (pairs.collect())
counts = pairs.reduceByKey(lambda a, b: a + b)
print (counts.collect())

['hello this is line one', 'hello this is line two']
['hello', 'this', 'is', 'line', 'one', 'hello', 'this', 'is', 'line', 'two']
[('hello', 1), ('this', 1), ('is', 1), ('line', 1), ('one', 1), ('hello', 1), ('this', 1), ('is', 1), ('line', 1), ('two', 1)]
[('hello', 2), ('this', 2), ('is', 2), ('line', 2), ('one', 1), ('two', 1)]


Локальные defs внутри функции, вызывающей Spark, для более длинного кода.
Функции верхнего уровня в модуле.
Например, чтобы передать более длинную функцию, чем может
поддерживаться с помощью lambda, рассмотрим следующий код:

In [ ]:
def myFunc(s):
 word = s.split(" ")
 return len(word)
words = ["hello this is line one", "hello this is line two"]
count = sc.parallelize(words).map(myFunc)
print(count.collect())

[5, 5]


Рассмотрим наивную сумму элементов RDD ниже, которая может вести себя
по-разному в зависимости от того, происходит ли выполнение в одной и той
же JVM. Типичным примером этого является запуск Spark в local режиме ( --
master = local[n]) по сравнению с развертыванием приложения Spark в кластере
(например, с помощью spark-submit в YARN):


In [ ]:
data = [1, 2, 3, 4, 5]
accum = sc.accumulator(0)
rdd = sc.parallelize(data)
def increment_counter(x):
 accum.add(x)
rdd.foreach(increment_counter)
print("Counter value: ", accum.value)

Counter value:  15
